Importing Libraries

In [26]:
import time
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud
import os
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Mounting to Google Drive Folder

In [27]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/DBA3803')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/DBA3803'

Reading training data and displaying a few rows of the training data

In [28]:
train = pd.read_csv("train.csv")
print(len(train))
train.head(5)

20000


,Comments,Insulting,Anti Government,Xenophobic,Racist,Sexual
0,FUCK FOREIGN TALENT,1,0,1,0,0
1,I LOVE THE GOVERNMENT,0,0,0,0,0
2,WOMEN SHOULD BE IN THE KITCHEN,1,0,0,0,0
3,I still don't know how he was crushed. What we...,1,1,0,0,0
4,In the first place how did a doctor become a d...,1,1,0,0,0


Creating a list for all the labels

In [0]:
label_cols = ['Insulting', 'Anti Government', 'Xenophobic', 'Racist', 'Sexual']

Using the CountVectorizer function to create a Bag of Words Document-Term Matrix using Unigrams only .


Displaying some of the features in the Matrix

In [30]:
print(len(train["Comments"]))
train = train.dropna()
vec = CountVectorizer()
trn_term_doc = vec.fit_transform(train["Comments"])
print(vec.get_feature_names()[10000:10050])
print(len(vec.get_feature_names()))

20000
['kallang', 'kam', 'kamakazi', 'kamijou', 'kamjia', 'kamlaojiao', 'kampong', 'kampung', 'kamu', 'kan', 'kana', 'kanasai', 'kang', 'kangaroo', 'kangkongfried', 'kangxi', 'kanina', 'kaninabe', 'kanji', 'kanna', 'kanning', 'kanojo', 'kanphua', 'kanpua', 'kantong', 'kanz', 'kao', 'kaobei', 'kaohsiung', 'kaopeh', 'kaoz', 'kap', 'kapo', 'kaput', 'kar', 'kara', 'karaoke', 'karcher', 'karchng', 'kardon', 'karen', 'karma', 'karmacheck', 'kat', 'kate', 'katib', 'katong', 'kawaii', 'kay', 'kayak']
20590


Sample of the Bag of Words Document-Term Matrix

In [31]:
x = trn_term_doc
df = pd.SparseDataFrame(x, columns = vec.get_feature_names())
df.loc[:5,["government","foreign","minister"]]

,government,foreign,minister
0,NaN,1.0,NaN
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,1.0
5,NaN,NaN,2.0


Creating a function where it creates a Logistic Regression model for each of the 5 labels with each label having their own y variable (either 1 or 0) but using the same x variable (Bag of Words Document-Term matrix)

In [0]:
scores = []
sd = []

def get_mdl(y):
    y = y.values
    m = LogisticRegression(solver = "liblinear")
    cv_score = np.mean(cross_val_score(
        m, x, y, cv=5, scoring='roc_auc'))
    cv_sd = np.std(cross_val_score(
        m, x, y, cv=5, scoring='roc_auc'))
    scores.append(cv_score)
    sd.append(cv_sd)
    print('CV score for class {} is {}, standard deviation is {}'.format(j, cv_score, cv_sd))
    return m.fit(x, y)
  
  

Running earlier functions to create 5 different logistic regression models representing each of the 5 labels.

Using 5-fold Cross Validation scoring by the Receiver Operating Characteristic, Area Under Curve measure we can gather the mean of the score and the mean of the standard deviation for each label. With these 5 scores and standard deviations we mean them once again to get the CV score and standard deviation that is representative of all labels.

In [33]:
for i, j in enumerate(label_cols):
    print('fit', j)
    get_mdl(train[j])
    
print('Total CV score is {}'.format(np.mean(scores)))

print('The Standard Deviation of the CV score is {}'.format(np.std(scores)))


fit Insulting
CV score for class Insulting is 0.6912120137111166, standard deviation is 0.057822115362626145
fit Anti Government
CV score for class Anti Government is 0.8384981726811256, standard deviation is 0.06828929918691057
fit Xenophobic
CV score for class Xenophobic is 0.8209293905600307, standard deviation is 0.0666651354867022
fit Racist
CV score for class Racist is 0.826231613691939, standard deviation is 0.03696109084415796
fit Sexual
CV score for class Sexual is 0.7848100073316072, standard deviation is 0.05436569069571951
Total CV score is 0.7923362395951637
The Standard Deviation of the CV score is 0.053628615432855645


Creating similar function as the one earlier, except without CV scores and standard deviations

In [0]:
def test_mdl(y):
    y = y.values
    m = LogisticRegression(solver = "liblinear", max_iter = 1000)
    return m.fit(x, y)

Running a test comment on the model and getting the run time for the test comment's results to be generated

In [35]:
start = time.time()

test_str = "You are stupid"
test_str = clean_text([test_str])
test_com = pd.Series(test_str)
test_vec = vec.transform(pd.Series(test_com))
test_pred = np.zeros((len(test_com), len(label_cols)))

for i, j in enumerate(label_cols):
    m = test_mdl(train[j])
    test_pred[:,i] = m.predict_proba(test_vec)[:,1]
    if test_pred[:,i] >= 0.5:
      print("\n","This comment is",j)
    
    
score = pd.DataFrame(test_pred, columns = label_cols)
print("\n\n",score)

end = time.time()
print("\n\n","The run time is:",end-start)

Preprocessing Begins...
['stupid'] 



    Insulting  Anti Government  Xenophobic    Racist    Sexual
0   0.453323         0.010362    0.008681  0.009161  0.031861


 The run time is: 1.556027889251709
